In [ ]:
import sys
sys.path.append('..')
from configure.settings import DBSelector
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
from pyecharts.charts import Scatter
import pyecharts.options as opts
from pyecharts.charts import Line, Grid
from pyecharts.commons.utils import JsCode
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import tree,preprocessing
from sklearn.feature_extraction import DictVectorizer
%matplotlib inline

background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
)
area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
)

In [ ]:
def get_bondPremRatio(conv_price,current_price):
    inner_value = get_conv_value(conv_price,current_price)
    return (100/inner_value-1)*100

def convert_permratio_inner(bondPremRatio):
    return 100/(bondPremRatio/100+1)

# 转股价值
def get_conv_value(conv_price,current_price):
    return 100/conv_price*current_price

In [ ]:
get_bondPremRatio(4.05,3.95)

In [ ]:
# load data
file = '/home/xda/hub/convertible_bond/data/new_bond_info.xlsx'
data = pd.read_excel(file,index_col=0)

In [ ]:
data.drop('neg_market_value',axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
rate_map = {
    'A':0,
    'A+':1,
    'AA-':2,
    'AA':3,
    'AA+':4,
    'AAA':5
}

data['rate'] = data['rate'].map(lambda x:rate_map.get(x))

In [ ]:
industry_map = {
'化工':0,
'医药生物':    1,
'机械设备':    2,
'电子':      3,
'电气设备':   4,
'轻工制造':    5,
'汽车':      6,
'有色金属':   7,
'计算机':     8,
'建筑装饰':    9,
'公用事业':    10,
'农林牧渔':    11,
'银行':    12,
'纺织服装':    13,
'食品饮料':    14,
'国防军工':     15,
'通信':       16,
'非银金融':     17,
'交通运输':     18,
'家用电器':     19,
'建筑材料':     20,
'采掘':       21,
'钢铁':       22,
'传媒':       23,
'商业贸易':    24,
'休闲服务':  25,
}
data['industry']=data['industry'].map(lambda x:industry_map.get(x))

In [ ]:
data['innerValue'] = data['bondPremRatio'].map(lambda x:convert_permratio_inner(x))

In [ ]:
train_data = data.copy()
train_data.drop('secShortNameBond',inplace=True,axis=1)
train_data.drop('bondPremRatio',inplace=True,axis=1)

data_source = train_data.copy()
target = data_source['closePriceBond']
data_source.drop('closePriceBond',axis=1,inplace=True)
data_source.drop('firstDate',axis=1,inplace=True)

In [ ]:
data_source.isnull().any()

In [ ]:
data_source['put_trigger_date'].fillna(0,inplace=True)

In [ ]:
data_source['put_trigger_rate'].fillna(0,inplace=True)

In [ ]:
data_source.drop('code',axis=1,inplace=True)

In [ ]:
data_source[data_source['industry'].isnull()]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data_source,target,random_state=4)

In [ ]:
rfr = RandomForestRegressor()

In [ ]:
from pprint import pprint

In [ ]:
pprint(rfr.get_params())

In [ ]:
rf_param_grid ={
    # 'criterion':['entropy','gini'],
    'max_depth':[5,6,7,8],
    'n_estimators':[11,13,15],
    # 'max_features':[0.3,0.4,0.5],
    'min_samples_split':[4,8,12,16]
}

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
import sklearn.metrics as metrics
metrics.classification_report(y_test,y_pred)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 150, num = 10)]
# Number of features to consider at every split
max_features = [3,4,5,6,6]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(3, 10, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3,4,5,6]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
rf = RandomForestRegressor()
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rfsv = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 200, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rfsv.fit(X_train,y_train)

In [ ]:
rfsv.best_params_

In [ ]:
rf4=RandomForestRegressor(**rfsv.best_params_)

In [ ]:
rf4.fit(X_train,y_train)

In [ ]:
y_pred4 = rf4.predict(X_test)

In [ ]:
evaluate_(y_test,y_pred4)

In [ ]:
y_pred = rfsv.predict(X_test)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
evaluate(rfsv,X_test,y_test)

In [ ]:
def evaluate_(y_test,y_test_pre):
    import sklearn.metrics as sm
    print('mean absolute error=',round(sm.mean_absolute_error(y_test,y_test_pre),2))
    print('mean squared error=',round(sm.mean_squared_error(y_test,y_test_pre),2))
    print('median absolute error=',round(sm.median_absolute_error(y_test,y_test_pre),2))
    print('explained variance score=',round(sm.explained_variance_score(y_test,y_test_pre),2))
    print('R2 score=',round(sm.r2_score(y_test,y_test_pre),2))

    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

In [ ]:
evaluate_(y_test,y_pred)

In [ ]:
rfsv.best_params_

In [ ]:
rfsv.feature_importances_

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10],
    'max_features': [4,5,6],
    'min_samples_leaf': [2,3, 4, 5,6],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [30]
}
# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
y_pred_grid = grid_search.predict(X_test)

In [ ]:
rf_ = RandomForestRegressor(**grid_search.best_params_)
rf_.fit(X_train,y_train)
y_pred_grid = rf_.predict(X_test)
evaluate_(y_test,y_pred_grid)

In [ ]:
evaluate_(y_test,y_pred_grid)

In [ ]:

importances = rf_.feature_importances_
importances=list(map(lambda x:round(x*100,0),importances))
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %.2f" % (f + 1, 30, X_train.columns[indices[f]], importances[indices[f]]))


In [ ]:
X_train.shape[1]

In [ ]:
grid_search.best_params_

In [ ]:
rf_ = RandomForestRegressor(
    n_estimators=50,
    max_depth=110,
    max_features=3,
    bootstrap=True,
    min_samples_split=8,
    min_samples_leaf=3
)

In [ ]:
feature_list_ = list(feature_list[0].keys())

In [ ]:
len(importances)

In [ ]:
X_train.shape

In [ ]:
get_bondPremRatio(9.02,13.90)

In [ ]:

input_data = {
'innerValue':convert_permratio_inner(2.5316455696202667),
'call_trigger_date':30,
'call_trigger_rate':130, # 强赎
'coupon':0.2,
'industry':industry_map.get('公用事业'),
# 'neg_market_value':21042642880,
'put_trigger_date':30,
'put_trigger_rate':70, # triggerRate 回售触发票面比例（%）
'rate':rate_map.get('AA+'),
'remainSize':30.00*10**9,
}

In [ ]:
df_real = pd.DataFrame([input_data])

In [ ]:
df_real

In [ ]:
rf_.predict(df_real)[0]

In [ ]:
rf4.feature_importances_

In [ ]:
100/(-35.107914/100+1)

In [ ]:
convert_permratio_inner(-35.107914)

In [ ]:
data_source.head()

In [ ]:
target.head()

In [ ]:
data_source.columns

In [ ]:
feature_list=[]
label_list =[]
feature_name = data_source.columns
label_name = target
for index,row in data_source.iterrows():
    feature={}
    for i in range(len(feature_name)):
        feature[feature_name[i]]=row[feature_name[i]]
    feature_list.append(feature)
    # label_list.append(row[label_name])

In [ ]:
feature_list[0]

In [ ]:
len(feature_list)

In [ ]:
len(data_source)

In [ ]:
feature_list[0]

In [ ]:
vec = DictVectorizer()
X = vec.fit_transform(feature_list).toarray()

In [ ]:
vec = DictVectorizer(sparse=False)
X = vec.fit_transform(data_source.to_dict(orient='records'))

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,label_list,random_state=4)

In [ ]:
#-*- coding:utf-8 -*-

#定义一组字典列表，用来表示多个数据样本（每个字典代表一个数据样本）
measurements = [{'city':1,'country':'CN','temperature':33.},{'city':2,'country':'UK','temperature':12.},{'city':3,'country':'USA','temperature':18.}]
#从sklearn.feature_extraction导入DictVectorizer
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
#输出转化后的特征矩阵
print(vec.fit_transform(measurements).toarray())
#输出各个维度的特征含义
print(vec.get_feature_names())


In [ ]:
with open('new_bond_tree.dot','w') as f:
    tree.export_graphviz(rf_.estimators_[0],feature_names=list(data_source.columns),out_file=f)

In [ ]:
import joblib
joblib.dump(rf_, "train_model.m")
# clf = joblib.load("train_model.m") ＃调用
